In [87]:
import os
import json
import pandas as pd
import traceback

In [88]:
from langchain.chat_models import ChatOpenAI

In [89]:
from dotenv import load_dotenv
load_dotenv()  # Take environment variables 

True

In [90]:
KEY = os.getenv("OPENAI_API_KEY")

In [91]:
llm = ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo", temperature = 0.5 )


In [92]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [93]:
# Next steps is to design the input and output prompt templates

# Step-1 The RESPONSE_JSON

RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
     "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
     "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [94]:
# Step -2 The input prompt template 

TEMPLATE = """
Text:{text}
Eres un creador experto de MCQ. Dado el texto anterior, es tu trabajo \
crear un cuestionario de {number} preguntas de opción múltiple para {subject} estudiantes en {tone} tone.
Make sure to format your response like RESPONSE_JSON bellow an use it as a guide. \
Asegurate de hacer {number} MCQs
{response_json}
"""

In [95]:
# Step-3 The input prompt with the variables that the user is going to pass

quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "respose_json"],
    template = TEMPLATE
)

In [96]:
# Step-4 Lets to create the chain to connect altogether

quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [97]:
# Step-5 THis is the second template to handle the correct answer

TEMPLATE2 = """ 
Eres un experto gramático y escritor. Se realizó una prueba de opción múltiple para estudiantes de {subject}. \
Debe evaluar la complejidad de la pregunta y realizar un análisis completo del cuestionario. Utilice únicamente un máximo de 50 palabras para mayor complejidad.
si el cuestionario no se ajusta a las capacidades cognitivas y analíticas de los estudiantes, \
actualice las preguntas del cuestionario que deben cambiarse y cambie el tono para que se ajuste perfectamente a las habilidades del estudiante
Preguntas frecuentes del cuestionario:
{quiz}

Verifique con un escritor experto en inglés el cuestionario anterior:
"""

In [98]:
# The chain evaluation

quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [99]:
# The quiz evaluation chain

review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [100]:
# Next we are going to connect both chains with SequentiaChain

generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                          output_variables = ["quiz", "review"], verbose=True)

In [101]:
# We need to provide data with some file path
file_path = r'C:\Users\Fernando Navarro\mcqgen\datos.txt'

In [102]:
# We need to provide data with some file 

with open(file_path, 'r') as file:
    TEXT = file.read()


In [103]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [104]:
# Define values for the generation. This are the input prametrer needed to create MCQs
NUMBER = 5
SUBJECT = "WW-II History"
TONE = "simple"

In [105]:
# The final step. FOr additional info:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage-tracking 

# Seting up:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )





> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:La Operación Windsor fue una ofensiva canadiense lanzada como parte de la batalla de Normandía durante la Segunda Guerra Mundial. El ataque, que tuvo lugar en los días 4 y 5 de julio de 1944, fue realizado por la 3.ª División de Infantería Canadiense en un intento de capturar la ciudad normanda de Carpiquet y el adyacente aeródromo de las fuerzas alemanas. A pesar de que fue planeado originalmente para ser realizado durante las últimas etapas de la Operación Epsom, como un medio para proteger el frente oriental del asalto principal,2​ fue pospuesto y lanzado a la semana siguiente.

El 4 de julio de 1944, cuatro batallones de la 3.ª División de Infantería Canadiense atacaron Carpiquet en conjunción con ataques envolventes de regimientos armados de la 2.ª Brigada Acorazada Canadiense. Aunque la 8.ª Brigada de Infantería Canadiense capturó exitosamente Carpiquet a media tarde, la re

In [106]:
# Show Counters 
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:8769
Prompt Tokens:7766
Completion Tokens:1003
Total Cost:0.013655


In [107]:
response

{'text': "La Operación Windsor fue una ofensiva canadiense lanzada como parte de la batalla de Normandía durante la Segunda Guerra Mundial. El ataque, que tuvo lugar en los días 4 y 5 de julio de 1944, fue realizado por la 3.ª División de Infantería Canadiense en un intento de capturar la ciudad normanda de Carpiquet y el adyacente aeródromo de las fuerzas alemanas. A pesar de que fue planeado originalmente para ser realizado durante las últimas etapas de la Operación Epsom, como un medio para proteger el frente oriental del asalto principal,2\u200b fue pospuesto y lanzado a la semana siguiente.\n\nEl 4 de julio de 1944, cuatro batallones de la 3.ª División de Infantería Canadiense atacaron Carpiquet en conjunción con ataques envolventes de regimientos armados de la 2.ª Brigada Acorazada Canadiense. Aunque la 8.ª Brigada de Infantería Canadiense capturó exitosamente Carpiquet a media tarde, la resistencia alemana al sur evitó la captura del aeródromo a pesar del significante blindaje a

In [108]:
quiz = response.get("quiz")

In [109]:
quiz = json.loads(quiz)

In [110]:
# Preparing the quiz for a better format

quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [111]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,¿Cuál fue el objetivo principal de la Operació...,a: Capturar la ciudad de Caen | b: Tomar el co...,b
1,¿Qué brigada de infantería canadiense capturó ...,a: 7.ª Brigada de Infantería Canadiense | b: 8...,b
2,¿Qué tipo de apoyo aéreo recibieron las fuerza...,a: Bombardeo de la Luftwaffe | b: Escuadrones ...,b
3,¿Cuál fue el resultado de los contraataques al...,a: Captura exitosa del aeródromo por parte de ...,c
4,¿Cuántas víctimas canadienses se reportaron co...,a: 377 | b: 127 | c: 155 | d: 130,a


In [112]:
# Se puede concertir tambien a csv
quiz = pd.DataFrame(quiz_table_data)
quiz.to_csv("WWIIQuiz.csv")